# `holodeck` - librarian

Module for generating and managing simulation libraries.  Holodeck libraries are groups of simulations in which certain parameters are varied, for example parameters concerning the masses of black holes (e.g. through the M-Mbulge relationship), or parameters dictating the rate of binary evolution.  For more information, see the [holodeck getting started guide](https://holodeck-gw.readthedocs.io/en/main/getting_started/index.html), and specifically the page on [holodeck libraries](https://holodeck-gw.readthedocs.io/en/main/getting_started/libraries.html).

Currently, libraries are only implemented from semi-analytic models (SAMs), which use the [`holodeck.sams.sam.Semi_Analytic_Model`](https://holodeck-gw.readthedocs.io/en/main/api_ref/holodeck.sams.html) class.  And binary evolution ('hardening') models implemented as subclasses of [`holodeck.hardening._Hardening`](https://holodeck-gw.readthedocs.io/en/main/api_ref/holodeck.hardening.html).

In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

import kalepy as kale

# --- Holodeck ----
import holodeck as holo
import holodeck.librarian
import holodeck.librarian.param_spaces
from holodeck import cosmo, utils, plot

log = holo.log
log.setLevel(log.WARNING)

## Parameter spaces – `_Param_Space` subclasses

Holodeck libraries are build around their parameter-spaces, which are implemented as subclasses of the [`holodeck.librarian.lib_tools._Param_Space`](https://holodeck-gw.readthedocs.io/en/main/api_ref/holodeck.librarian.lib_tools.html#holodeck.librarian.lib_tools._Param_Space) base class.  These subclasses should be named with `PS_` prefixes to denote that they are parameter spaces.

In [ ]:
PSPACE_CLASS = holo.librarian.param_spaces_dict['PS_Test']   #: Choose which parameter-space class to use
NSAMPLES = 10    #: Number of samples to draw
SAM_SHAPE = (20, 21, 22)   #: Shape of the semi-analytic model grid (in the `Semi_Analytic_Model` class)

In [ ]:
# construct an instance of the parameter-space class, which draws samples from the parameter space
# using a latin hypercube
pspace = PSPACE_CLASS(holo.log, nsamples=NSAMPLES, sam_shape=SAM_SHAPE)
print(f"pspace '{pspace.name}'")
print(f"\tlibrary shape={pspace.lib_shape} (samples, parameters)")
print(f"\tSAM grid shape={pspace.sam_shape}")
print(f"\tparameters ({pspace.nparameters}):")
for pp, dist, extr in zip(pspace.param_names, pspace._parameters, pspace.extrema):
    print(f"\t\t{pp} (distribution:{dist.__class__.__name__}): {extr}")

Create a corner plot showing where the samples are in each 1D and 2D slice of the parameter space.

In [ ]:
kale.corner(
    pspace.param_samples.T, labels=pspace.param_names,
    dist2d=dict(scatter=True, hist=False, contour=False),
    dist1d=dict(carpet=True, hist=True, density=False),
)
plt.show()

Build model instances (`sam` instance of `Semi_Analytic_Model` and `hard` instance of `_Hardening`)

In [ ]:
# The sample parameters are stored as an array `param_samples`, which has a shape of `(S, P)` for
# `S` random samples and `P` parameters
SAMP = 4
print(f"Parameters for sample {SAMP}: {pspace.param_samples[SAMP]}")
# We can also obtain a dictionary of these parameters, with the parameter names as keys:
params = pspace.param_dict(SAMP)
for kk, vv in params.items():
    print(f"\t{kk:20s}: {vv:+.2e}")

# We can load `Semi_Analytic_Model` and `_Hardening` class instances for this in two ways:
# (1) by passing in this dictionary of parameters:
sam, hard = pspace.model_for_params(params)
# (2) by passing in the sample number
sam, hard = pspace.model_for_sample_number(SAMP)

Run the model

In [ ]:
data = holo.librarian.lib_tools.run_model(sam, hard, details_flag=True)
print(data.keys())
data['gwb']

### Custom class

In [ ]:
class PS_Lib_Test(holo.librarian.lib_tools._Param_Space):

    DEFAULTS = {'gsmf_phi0_log10': -2.77}

    def __init__(self, log, nsamples, sam_shape):
        parameters = [
            holo.librarian.lib_tools.PD_Normal("gsmf_phi0_log10", -2.77, 0.3),
        ]
        super().__init__(
            parameters,
            log=log, nsamples=nsamples, sam_shape=sam_shape,
        )
        return

    @classmethod
    def _init_sam(cls, sam_shape, params):
        gsmf = holo.sams.components.GSMF_Schechter(phi0=params['gsmf_phi0_log10'])
        sam = holo.sams.sam.Semi_Analytic_Model(gsmf=gsmf, shape=sam_shape)
        return sam

    @classmethod
    def _init_hard(cls, sam, params):
        hard = holo.hardening.Hard_GW()
        return hard

test = PS_Lib_Test(log, 4, 10)
fobs_cents, fobs_edges = utils.pta_freqs()
sam, hard = test.model_for_sample_number(0)
hc_ss, hc_bg = sam.gwb(fobs_edges, hard=hard, realize=20)
plot.plot_gwb(fobs_cents, hc_bg)
plt.show()

## Parameter distributions – `_Param_Dist` subclasses

### Uniform (`PD_Uniform`)

In [ ]:
test = holo.librarian.lib_tools.PD_Uniform("test", -10.0, 10.0)
assert test(0.5) == 0.0
assert test(0.0) == -10.0
assert test(1.0) == +10.0

xx = np.linspace(0.0, 1.0, 100)
yy = test(xx)
plt.plot(xx, yy, 'k-')
plt.show()

### Normal (`PD_Normal`)

In [ ]:
test = holo.librarian.lib_tools.PD_Normal("test", 0.0, 1.0)
val = test(0.5)
print(val)
assert test(0.5) == 0.0

xx = np.linspace(0.0, 1.0, 100)
yy = test(xx)
plt.plot(xx, yy, 'k-')
plt.show()

### LinLog (`PD_Lin_Log`)

In [ ]:
test = holo.librarian.lib_tools.PD_Lin_Log("test", 0.01, 100.0, 0.1, 0.5)
xx = np.linspace(0.0, 1.0, 10000)
yy = test(xx)
print(utils.minmax(yy))
plt.loglog(xx, yy)
ax = plt.gca()
ax.axhline(test._crit, color='r', ls=':')
ax.axvline(test._lofrac, color='r', ls=':')
plt.show()

Change the fraction of population below/above cutoff

In [ ]:
NUM = int(1e4)
crit = 0.1

BINS = 20
e1 = np.linspace(0.01, crit, BINS, endpoint=False)
e2 = np.logspace(*np.log10([crit, 100.0]), BINS)
edges = np.concatenate([e1, e2])

fig, ax = plot.figax(scale='log')
for frac in [0.2, 0.5, 0.8]:
    test = holo.librarian.lib_tools.PD_Lin_Log("test", 0.01, 100.0, crit, frac)
    xx = test(np.random.uniform(0.0, 1.0, size=NUM))
    kale.dist1d(xx, ax=ax, edges=edges, density=True, probability=False)
    obs_frac = np.count_nonzero(xx < crit) / xx.size
    print(f"target:{frac:.2f}, result:{obs_frac:.4f}", 1.0/np.sqrt(NUM))
    assert np.isclose(frac, obs_frac, atol=2.0/np.sqrt(NUM))

plt.show()

Change the location of the cutoff

In [ ]:
NUM = int(1e4)
frac = 0.5

BINS = 20
edges = np.logspace(*np.log10([0.01, 100.0]), 2*BINS)

fig, ax = plot.figax(scale='log')
for crit in [0.1, 1.0, 10.0]:
    test = holo.librarian.lib_tools.PD_Lin_Log("test", 0.01, 100.0, crit, frac)
    xx = test(np.random.uniform(0.0, 1.0, size=NUM))
    kale.dist1d(xx, ax=ax, edges=edges, density=True, probability=False)
    obs_frac = np.count_nonzero(xx < crit) / xx.size
    print(f"target:{frac:.2f}, result:{obs_frac:.4f}", 1.0/np.sqrt(NUM))
    assert np.isclose(frac, obs_frac, atol=2.0/np.sqrt(NUM))

plt.show()

### LogLin (`PD_Log_Lin`)

In [ ]:
test = holo.librarian.lib_tools.PD_Log_Lin("test", 0.01, 100.0, 0.1, 0.5)
xx = np.linspace(0.0, 1.0, 10000)
yy = test(xx)
print(utils.minmax(yy))
plt.loglog(xx, yy)
ax = plt.gca()
ax.axhline(test._crit, color='r', ls=':')
ax.axvline(test._lofrac, color='r', ls=':')
plt.show()

Change the fraction of population below/above cutoff

In [ ]:
NUM = int(2e4)
crit = 0.1

BINS = 30
edges = np.logspace(*np.log10([0.01, 100.0]), BINS)

fig, ax = plot.figax(scale='log')
for frac in [0.2, 0.5, 0.8]:
    test = holo.librarian.lib_tools.PD_Log_Lin("test", 0.01, 100.0, crit, frac)
    xx = test(np.random.uniform(0.0, 1.0, size=NUM))
    kale.dist1d(xx, ax=ax, edges=edges, density=True, probability=False)
    obs_frac = np.count_nonzero(xx < crit) / xx.size
    print(f"target:{frac:.2f}, result:{obs_frac:.4f}", 1.0/np.sqrt(NUM))
    assert np.isclose(frac, obs_frac, atol=2.0/np.sqrt(NUM))

plt.show()

Change the location of the cutoff

In [ ]:
NUM = int(2e4)
frac = 0.5

BINS = 20

edges = np.logspace(*np.log10([0.01, 100.0]), 2*BINS)

fig, ax = plot.figax(scale='log')
for crit in [0.1, 1.0, 10.0]:
    test = holo.librarian.lib_tools.PD_Log_Lin("test", 0.01, 100.0, crit, frac)
    xx = test(np.random.uniform(0.0, 1.0, size=NUM))
    kale.dist1d(xx, ax=ax, edges=edges, density=True, probability=False)
    obs_frac = np.count_nonzero(xx < crit) / xx.size
    print(f"target:{frac:.2f}, result:{obs_frac:.4f}", 1.0/np.sqrt(NUM))
    assert np.isclose(frac, obs_frac, atol=2.0/np.sqrt(NUM))

plt.show()

### Piecewise Uniform in Mass (`PD_Piecewise_Uniform_Mass`)

In [ ]:
edges = [-1.0, 5.0, 6.0, 7.0]
amps = [1.0, 2.0, 1.0]
test = holodeck.librarian.lib_tools.PD_Piecewise_Uniform_Mass("test", edges, amps)

xx = np.random.uniform(size=1000)
xx = np.sort(xx)
yy = test(xx)
print(utils.minmax(yy))
x, y, _ = plt.hist(yy, histtype='step', density=True, bins=edges)
ax = plt.gca()
plt.show()

In [ ]:
edges = [-1.0, 4.0, 6.0, 7.5]
test = holodeck.librarian.lib_tools.PD_Piecewise_Uniform_Density("test", edges, [1.0, 2.0, 1.0])

xx = np.random.uniform(size=1000)
xx = np.sort(xx)
yy = test(xx)
print(utils.minmax(yy))
x, y, _ = plt.hist(yy, histtype='step', density=True, bins=edges)
ax = plt.gca()
plt.show()

### Piecewise Uniform in Density (`PD_Piecewise_Uniform_Density`)

In [ ]:
edges = [0.1, 1.0, 9.0, 11.0]
test = holodeck.librarian.lib_tools.PD_Piecewise_Uniform_Density("test", edges, [2.5, 0.5, 1.5])

xx = np.random.uniform(size=2000)
xx = np.sort(xx)
yy = test(xx)
print(utils.minmax(yy))
ax = plt.gca()
scale = 'log'
scale = 'linear'
ax.set(xscale=scale)
xx = kale.utils.spacing(edges, scale, num=20)
ax.hist(yy, histtype='step', density=True, bins=xx)
# tw = ax.twinx()
# tw.hist(yy, histtype='step', density=True, bins=30)
plt.show()

In [ ]:
test = holodeck.librarian.lib_tools.PD_Piecewise_Uniform_Density(
    "test", [7.5, 8.0, 9.0, 9.5], [1.5, 1.0, 2.0]
)

xx = np.random.uniform(size=2000)
xx = np.sort(xx)
yy = test(xx)
print(utils.minmax(yy))
ax = plt.gca()
scale = 'log'
scale = 'linear'
ax.set(xscale=scale)
xx = kale.utils.spacing(yy, scale, num=20)
print(xx)
ax.hist(yy, histtype='step', density=True, bins=xx)
# tw = ax.twinx()
# tw.hist(yy, histtype='step', density=True, bins=30)
plt.show()